# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [5]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import time
import torchvision.transforms as transforms
from tqdm import tqdm

BATCH_SIZE = 128
NUM_EPOCHS = 100

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [6]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [7]:
from torch.nn import functional as F

class Lenet(nn.Module):
    # Define model
    def __init__(self):
        super(Lenet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3, stride=1,padding=1)
        self.conv2 = nn.Conv2d(6, 16, 5, stride=1, padding=0)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # Convolution
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        
        # Full connected layer
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x
    
model = Lenet()

Next, we can start to train and evaluate!

In [9]:
from torch.autograd import Variable

def train_model(model, optimizer, criterion, epoch):
    model = model.cuda()

    # train and evaluate
    train_loss = 0
    correct_pred = 0
    for batchidx, (datas, labels) in enumerate(train_loader):
        # zero the parameter gradient
        optimizer.zero_grad()

        # load the data into cuda
        datas, labels = datas.cuda(), labels.cuda()
        datas, labels = Variable(datas), Variable(labels)

        # Forward, backward, optimize
        out = model(datas)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()

        # Add loss
        train_loss += loss.item()

        pred = out.argmax(dim=1, keepdim=True)
        correct_pred += pred.eq(labels.view_as(pred)).sum().item()
    
    train_accuracy = correct_pred/len(train_dataset)
    # Print training result
    print("Training Epoch:{} loss:{} train accuracy:{}".format(
        epoch, train_loss, train_accuracy))
    return train_accuracy

def test_model(model):
    # evaluate
    correct_test = 0
    test_loss = 0
    for batchidx, (datas, labels) in enumerate(test_loader):
        datas, labels = datas.cuda(), labels.cuda()
        out = model(datas)
        test_loss += criterion(out, labels)
        pred = out.argmax(dim=1, keepdim=True)
        correct_test += pred.eq(labels.view_as(pred)).sum().item()
    
    test_accuracy = correct_test/len(test_dataset)
    print("Test loss:{} test accuracy:{}".format(
        test_loss, test_accuracy))
    return test_accuracy
    
if __name__ == "__main__":
    model = Lenet()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.07)
    
    for i in range(NUM_EPOCHS):
        train_accuracy = train_model(model, optimizer, criterion, i)
        test_accuracy  = test_model(model)

Training Epoch:0 loss:479.659077167511 train accuracy:0.6756
Test loss:12.629231452941895 test accuracy:0.9479
Training Epoch:1 loss:55.733505349606276 train accuracy:0.9620833333333333
Test loss:5.43077278137207 test accuracy:0.9759
Training Epoch:2 loss:36.155685756355524 train accuracy:0.9739333333333333
Test loss:4.428620338439941 test accuracy:0.9812
Training Epoch:3 loss:27.769600056111813 train accuracy:0.98
Test loss:3.7956814765930176 test accuracy:0.9822
Training Epoch:4 loss:22.827697901055217 train accuracy:0.9834166666666667
Test loss:3.30773663520813 test accuracy:0.9853
Training Epoch:5 loss:19.558279015123844 train accuracy:0.9855666666666667
Test loss:3.0008373260498047 test accuracy:0.986
Training Epoch:6 loss:16.829195016995072 train accuracy:0.9869666666666667
Test loss:3.0981287956237793 test accuracy:0.9845
Training Epoch:7 loss:14.52556836232543 train accuracy:0.9885833333333334
Test loss:2.5836009979248047 test accuracy:0.9874
Training Epoch:8 loss:12.9988618399

Test loss:3.3473353385925293 test accuracy:0.9887
Training Epoch:67 loss:0.08440062031149864 train accuracy:0.9983833333333333
Test loss:3.348032236099243 test accuracy:0.9889
Training Epoch:68 loss:0.0789526216685772 train accuracy:0.9983833333333333
Test loss:3.3507893085479736 test accuracy:0.989
Training Epoch:69 loss:0.08016923069953918 train accuracy:0.9983833333333333
Test loss:3.3764171600341797 test accuracy:0.9887
Training Epoch:70 loss:0.07546404376626015 train accuracy:0.9983833333333333
Test loss:3.3739097118377686 test accuracy:0.9891
Training Epoch:71 loss:0.07455017045140266 train accuracy:0.9983833333333333
Test loss:3.389232635498047 test accuracy:0.9889
Training Epoch:72 loss:0.07177343219518661 train accuracy:0.9983833333333333
Test loss:3.413372278213501 test accuracy:0.989
Training Epoch:73 loss:0.07116541638970375 train accuracy:0.9983833333333333
Test loss:3.4159634113311768 test accuracy:0.989
Training Epoch:74 loss:0.06934407725930214 train accuracy:0.99838333

#### Q5:
Please print the training and testing accuracy.

In [10]:
print("train loss: {}".format(train_accuracy))
print("test loss: {}".format(test_accuracy))

train loss: 0.9983833333333333
test loss: 0.9889
